# SurrealDBVectorStore

This notebook covers how to get started with the SurrealDB vector store.

## Setup

- [Install SurrealDB](https://surrealdb.com/docs/surrealdb/installation) to run the DB locally.
- Start the DB in memory with:
    ```shell
    $ surreal start -u root -p root
    ```
    Find more options here [Running SurrealDB](https://surrealdb.com/docs/surrealdb/installation/running).

- Install `langchain-surrealdb` and `surrealdb` python packages.
    ```shell
    # -- Using pip
    pip install --upgrade langchain-surrealdb surrealdb
    # -- Using poetry
    poetry add langchain-surrealdb surrealdb
    # -- Using uv
    uv pip install --upgrade langchain-surrealdb surrealdb
    ```

To run this notebook, we just need to install the additional dependencies required by this example:


In [1]:
!poetry add --quiet --group docs langchain-ollama

## Initialization


In [2]:
from langchain_surrealdb.vectorstores import SurrealDBVectorStore
from langchain_ollama import OllamaEmbeddings
from surrealdb import Surreal

conn = Surreal("ws://localhost:8000/rpc")
conn.signin({"username": "root", "password": "root"})
conn.use("langchain", "demo")
vector_store = SurrealDBVectorStore(OllamaEmbeddings(model="llama3.2"), conn)

## Manage vector store

### Add items to vector store


In [3]:
from langchain_core.documents import Document

document_1 = Document(page_content="foo", metadata={"source": "https://example.com"})

document_2 = Document(page_content="bar", metadata={"source": "https://example.com"})

document_3 = Document(page_content="baz", metadata={"source": "https://example.com"})

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents, ids=["1", "2", "3"])

['1', '2', '3']

### Update items in vector store

In [4]:
updated_document = Document(
    page_content="qux", metadata={"source": "https://another-example.com"}
)

vector_store.add_documents(documents=[updated_document], ids=["1"])

['1']

### Delete items from vector store


In [5]:
vector_store.delete(ids=["3"])

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Query directly

Performing a simple similarity search can be done as follows:


In [6]:
results = vector_store.similarity_search(
    query="thud", k=1, custom_filter={"source": "https://another-example.com"}
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* qux [{'source': 'https://another-example.com'}]


If you want to execute a similarity search and receive the corresponding scores you can run:


In [7]:
results = vector_store.similarity_search_with_score(
    query="thud", k=1, custom_filter={"source": "https://example.com"}
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.430072] bar [{'source': 'https://example.com'}]


### Query by turning into retriever

You can also transform the vector store into a retriever for easier usage in your chains. 


In [8]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("thud")

[Document(id='1', metadata={'source': 'https://another-example.com'}, page_content='qux')]

## Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials](/docs/tutorials/)
- [How-to: Question and answer with RAG](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](https://python.langchain.com/docs/concepts/retrieval/)

## API reference

For detailed documentation of all SurrealDBVectorStore features and configurations head to the API reference: https://api.python.langchain.com/en/latest/vectorstores/langchain_surrealdb.vectorstores.SurrealDBVectorStore.html